In [3]:
import pandas as pd
import pyarc
from pyids import IDS
from pyids.data_structures.ids_classifier import mine_CARs
import scipy
from pyids.model_selection import mode
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score

In [4]:
from pyarc.qcba.transformation import QCBATransformation

from pyarc import CBA
from pyarc.data_structures import TransactionDB
from pyarc.qcba.data_structures import QuantitativeDataFrame
import pandas as pd

from pyarc.qcba.data_structures import (
    IntervalReader,
    Interval,
    QuantitativeDataFrame,
    QuantitativeCAR
)

from pyarc.qcba.classifier import QuantitativeClassifier
from sklearn.metrics import accuracy_score

interval_reader = IntervalReader()

interval_reader.closed_bracket = "", "NULL"
interval_reader.open_bracket = "NULL", ""
interval_reader.infinity_symbol = "inf", "inf"
interval_reader.members_separator = "_to_"

interval_reader.compile_reader()

QuantitativeCAR.interval_reader = interval_reader

In [98]:
data_train_disc = pd.read_csv("C:/code/python/machine_learning/assoc_rules/train/iris0.csv")
data_test_disc = pd.read_csv("C:/code/python/machine_learning/assoc_rules/train/iris1.csv")

In [99]:
quant_dataframe_train_disc = QuantitativeDataFrame(data_train_disc)
quant_dataframe_test_disc = QuantitativeDataFrame(data_test_disc)

In [117]:
cars = mine_CARs(data_train_disc, 30)

Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=5
Rule count: 96, Iteration: 1
Increasing maxlen 4
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=4, MAX_RULE_LEN=5
Rule count: 166, Iteration: 2
Increasing maxlen 5
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 190, Iteration: 3
Decreasing confidence to 0.45
Running apriori with setting: confidence=0.45, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 190, Iteration: 4
Decreasing confidence to 0.4
Running apriori with setting: confidence=0.4, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 190, Iteration: 5
Decreasing confidence to 0.35000000000000003
Running apriori with setting: confidence=0.35000000000000003, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 190, Iteration: 6
Decreasing confidence to 0.30000000000000004
Running apriori with setting: confidence=0.30000000

In [118]:
ids = IDS()
ids.fit(class_association_rules=cars, quant_dataframe=quant_dataframe_train_disc, debug=False, default_class="ma")

cover cache prepared
overlap cache prepared


In [119]:
ids.clf.rules

[IDS-CAR {petallength=-inf_to_2.45,petalwidth=-inf_to_0.8} => {class=Iris-setosa} sup: 0.33 conf: 1.00 len: 3, id: 3308 f1: 0.5,
 IDS-CAR {petallength=-inf_to_2.45,sepallength=-inf_to_5.55} => {class=Iris-setosa} sup: 0.31 conf: 1.00 len: 3, id: 3297 f1: 0.4745762711864407,
 IDS-CAR {petalwidth=1.75_to_inf,sepallength=6.25_to_inf} => {class=Iris-virginica} sup: 0.24 conf: 1.00 len: 3, id: 3349 f1: 0.38323353293413176,
 IDS-CAR {petalwidth=1.75_to_inf,sepallength=6.25_to_inf,petallength=4.75_to_inf} => {class=Iris-virginica} sup: 0.24 conf: 1.00 len: 4, id: 3348 f1: 0.38323353293413176,
 IDS-CAR {sepalwidth=3.35_to_inf,petalwidth=-inf_to_0.8} => {class=Iris-setosa} sup: 0.20 conf: 1.00 len: 3, id: 3401 f1: 0.3333333333333333,
 IDS-CAR {sepalwidth=3.35_to_inf,petallength=-inf_to_2.45,sepallength=-inf_to_5.55} => {class=Iris-setosa} sup: 0.18 conf: 1.00 len: 4, id: 3402 f1: 0.3018867924528302,
 IDS-CAR {sepalwidth=2.95_to_3.35,petalwidth=1.75_to_inf,sepallength=6.25_to_inf} => {class=Iris

In [122]:
print("Přednost IDS (train):", ids.score(quant_dataframe_train_disc))
print("Přednost IDS (test):", ids.score(quant_dataframe_test_disc))

Přednost IDS (train): 0.6814814814814815
Přednost IDS (test): 0.5703703703703704


In [123]:
ids.predict(quant_dataframe_train_disc)

['Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-versicolor',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-setosa',
 'Iris-versicolor',
 'Iris-versicolor',
 'Ir

In [72]:
#iris test dataset obsahuje 1/3 instancí každé třídy, tedy výsledek test "Přednost IDS (test): 0.3333333333333333" odpovídá situaci, kdy se použil pouze modus.

In [42]:
#dataframe 
quant_dataframe_test_disc.dataframe.iloc[40:100, :]
#pro instanci 6 je predikována třída "setosa" ačkoliv žádné z pravidel ve výpisu výše predikující setosa nelze použít (jestli jsem se dobře díval)
# lze ale použít pravidlo predikující Iris-versicolor (id 476)

,sepallength,petalwidth,sepalwidth,petallength,class
40,-inf_to_5.55,-inf_to_0.8,-inf_to_3.05,-inf_to_2.45,Iris-setosa
41,-inf_to_5.55,-inf_to_0.8,3.05_to_inf,-inf_to_2.45,Iris-setosa
42,-inf_to_5.55,-inf_to_0.8,3.05_to_inf,-inf_to_2.45,Iris-setosa
43,-inf_to_5.55,-inf_to_0.8,3.05_to_inf,-inf_to_2.45,Iris-setosa
44,-inf_to_5.55,-inf_to_0.8,3.05_to_inf,-inf_to_2.45,Iris-setosa
45,6.25_to_inf,0.8_to_1.75,3.05_to_inf,2.45_to_4.85,Iris-versicolor
46,6.25_to_inf,0.8_to_1.75,3.05_to_inf,2.45_to_4.85,Iris-versicolor
47,6.25_to_inf,0.8_to_1.75,3.05_to_inf,4.85_to_inf,Iris-versicolor
48,-inf_to_5.55,0.8_to_1.75,-inf_to_3.05,2.45_to_4.85,Iris-versicolor
49,6.25_to_inf,0.8_to_1.75,-inf_to_3.05,2.45_to_4.85,Iris-versicolor


In [ ]:
# výpis níže ukazuje, že všechny instance jsou skutečně predikovány jako setosa -stejně jako výchozí třída.

In [38]:
#PREDICT FUNCTION FOR DEBUGGING

quant_dataframe = quant_dataframe_test_disc
if type(quant_dataframe) != QuantitativeDataFrame:
    print("Type of quant_dataframe must be QuantitativeDataFrame")


Y = quant_dataframe.dataframe.iloc[:,-1]
y_pred_dict = dict()

for rule in ids.clf.rules:

    conf = rule.car.confidence
    sup = rule.car.support

    y_pred_per_rule = rule.predict(quant_dataframe)
    rule_f1_score = scipy.stats.hmean([conf, sup])

    y_pred_dict.update({rule_f1_score: y_pred_per_rule})


# rules in rows, instances in columns
y_pred_array = np.array(list(y_pred_dict.values()))
y_pred = []

minority_classes = []

if y_pred_dict:
    for i in range(len(Y)):
        all_NA = np.all(y_pred_array[:,i] == IDSRule.DUMMY_LABEL)
        if all_NA:
            minority_classes.append(Y[i])

    # if the ruleset covers all instances                     
    default_class = Y[0]

    if minority_classes:
        default_class = mode(minority_classes)

    for i in range(len(Y)):
        y_pred_array_datacase = y_pred_array[:,i]
        non_na_mask = y_pred_array_datacase != IDSRule.DUMMY_LABEL

        y_pred_array_datacase_non_na = y_pred_array_datacase[non_na_mask]

        if len(y_pred_array_datacase_non_na) > 0:
            y_pred.append(y_pred_array_datacase_non_na[0])
        else:
            y_pred.append(default_class)

else:
    y_pred = len(Y) * [mode(Y)]

print(y_pred)
print(default_class)

['Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa']
Iris-setosa
